In [1]:
'''
    author: Boyang Yan
    date:2022/10/13
    code for ranking task for our model
    
'''

'\n    author: Boyang Yan\n    date:2022/10/13\n    code for ranking task for our model\n    \n'

In [2]:
'''
    import part
    
'''

import scipy.io as scio
import numpy as np
import pandas as pd
from collections import defaultdict
import pickle

In [3]:
#data文件的放置路径 Ciao/Epinions
workdir = 'data/ciao'
#workdir = 'data/epinions'

#scio.loadmat读取数据集 
#user-item交互网络
rating = scio.loadmat(workdir + '/rating.mat')
#定位到rating mat的rating数据中
rating_f = rating['rating']
#pandas  处理
rating_dt = pd.DataFrame(rating_f)
rating_dt = rating_dt[[0,1,3]]
rating_dt.columns = ['userID','itemID','rating']
rating_dt.dropna(axis=0, how='any', inplace=True)
rating_dt.drop_duplicates(inplace=True)
print(rating)
print(rating_f)
print(rating_dt)

#社交网络
trust = scio.loadmat(workdir + '/trustnetwork.mat')
#定位到trust mat的trustnetwork数据中
trust_f = trust['trustnetwork']
#pandas 处理
trust_dt = pd.DataFrame(trust_f, columns=['userID', 'friendID'])
#dropna inplace=True时，表示直接修改原数据,axis=0表示删除包含缺失值的行，默认为0，axis=1时，表示删除包含缺失值的列；how=any表示只要存在缺失值就直接删除行或者列
trust_dt.dropna(axis=0, how='any', inplace=True)
#drop_duplicates表示去除重复行，inplace表示在原数据上修改
trust_dt.drop_duplicates(inplace=True)
print(trust)
print(trust_f)
print(trust_dt)


{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Fri Feb 24 17:28:03 2012', '__version__': '1.0', '__globals__': [], 'rating': array([[     1,      1,      1,      3,      1],
       [     1,      2,      1,      4,      1],
       [     1,      3,      2,      4,      1],
       ...,
       [  7375,  72772,     24,      4,      2],
       [  7375,  74985,     24,      5,      1],
       [  7375, 101247,     24,      1,      4]])}
[[     1      1      1      3      1]
 [     1      2      1      4      1]
 [     1      3      2      4      1]
 ...
 [  7375  72772     24      4      2]
 [  7375  74985     24      5      1]
 [  7375 101247     24      1      4]]
        userID  itemID  rating
0            1       1       3
1            1       2       4
2            1       3       4
3            1       4       5
4            1       5       4
...        ...     ...     ...
284081    7375   73419       1
284082    7375   25291       3
284083    7375   72772       4
2

In [4]:
'''去掉没有社会关系的用户'''
delete = []
print(len(trust_f))
for i in range(len(trust_f)):
    if trust_f[i,0] == trust_f[i,1]:
        print(i)
        delete.append(i)
        print(i)
trust_f = np.delete(trust_f,delete,0)

print(len(trust_f))
'''
numpy.delete(arr, obj, axis)

arr：需要处理的矩阵
obj：在什么位置处理
axis：这是一个可选参数，axis = None，1，0

axis=None：arr会先按行展开，然后按照obj，删除第obj-1（从0开始）位置的数，返回一个行矩阵。
axis = 0：arr按行删除
axis = 1：arr按列删除
所有操作都是在arr的副本进行，需要有变量接收返回值。

'''



#test vaild train训练集的划分比例，prob表示test和vaild的比例，默认0.2，即2 2 6H划分
prob = 0.2




111781
111781


In [5]:
'''用户,评分，物品初始矩阵构建'''
dic_user_o2i = {}
i = 0
# build user index
for user in trust_f.reshape(-1,):
  if user in dic_user_o2i.keys():
    continue
  else:
    dic_user_o2i[user] = i
    i += 1
print('building from trust_f',len(dic_user_o2i))

print('删除不存在user链接的item前的rating_num:',len(rating_f))
delete = []
for i in range(len(rating_f)):
  if rating_f[i,0] not in dic_user_o2i.keys():
    delete.append(i)
rating_f = np.delete(rating_f, delete, 0)
print('删除不存在user链接的item后的rating_num:',len(rating_f))

rating_dt = pd.DataFrame(rating_f)
rating_dt = rating_dt[[0,1,3]]
rating_dt.columns = ['userID','itemID','rating']
rating_dt.dropna(axis=0, how='any', inplace=True)
rating_dt.drop_duplicates(inplace=True)

print(rating_dt)

dic_item_o2i = {}
i = 0
for item in rating_f[:, 1]:
  if item in dic_item_o2i.keys():
    continue
  else:
    dic_item_o2i[item] = i
    i += 1
print('number of items ', len(dic_item_o2i.keys()))

building from trust_f 7317
删除不存在user链接的item前的rating_num: 284086
删除不存在user链接的item后的rating_num: 283320
        userID  itemID  rating
0            1       1       3
1            1       2       4
2            1       3       4
3            1       4       5
4            1       5       4
...        ...     ...     ...
283315    7375   73419       1
283316    7375   25291       3
283317    7375   72772       4
283318    7375   74985       5
283319    7375  101247       1

[282364 rows x 3 columns]
number of items  104975


In [6]:
#print(rating_f[0][5])
#阈值T矩阵
avg_user = rating_dt.groupby(['userID'])['rating'].mean()
print(avg_user)
#判断rating是否大于阈值（该用户的评分的平均值再取整数）
for i in range(len(rating_f)):
    if(rating_f[i][3] >= round(avg_user[rating_f[i][0]])):
        rating_f[i][4] = 1.0
    elif(rating_f[i][3] < round(avg_user[rating_f[i][0]])):
        rating_f[i][4] = 0.0
        
#print(rating_f[748][4])
#print(round(avg_user[2]))

userID
1       4.499318
2       4.700000
3       4.913043
4       4.424084
5       4.173913
          ...   
7371    4.375000
7372    3.071429
7373    4.500000
7374    4.700000
7375    3.000000
Name: rating, Length: 7317, dtype: float64


In [7]:
history_u_lists = defaultdict()
history_ur_lists = defaultdict()
history_v_lists = defaultdict()
history_vr_lists = defaultdict()
social_adj_lists = defaultdict()
item_adj_lists = defaultdict()
ratings_list = defaultdict()


    

data_total = []
print('building social_adj_lists')
for user in range(len(dic_user_o2i)):
  social_adj_lists[user] = []
for line in trust_f:
  social_adj_lists[dic_user_o2i[line[0]]].append(dic_user_o2i[line[1]])
  social_adj_lists[dic_user_o2i[line[1]]].append(dic_user_o2i[line[0]])
#print(len(dic_user_o2i)) #7317个用户
for user in range(len(dic_user_o2i)):
  social_adj_lists[user] = set(social_adj_lists[user])

print(social_adj_lists[7316])

print('building ratings_list')
i = 0
for rate in set(rating_f[:, 3]):
  ratings_list[rate] = i
  i += 1
  
print('building other dicts')
for user in range(len(dic_user_o2i)):
  history_u_lists[user] = []
  history_ur_lists[user] = []
for item in range(len(dic_item_o2i)):
  history_v_lists[item] = []
  history_vr_lists[item] = []
  

#user, item, rating, label
rating_f = rating_f[:, [0, 1, 3, 4]]
np.random.shuffle(rating_f)
valid = rating_f[ : int(0.2 * len(rating_f))]
test = rating_f[int(0.2 * len(rating_f)) : int(0.4 * len(rating_f))]
train = rating_f[int(0.4 * len(rating_f)): ]

building social_adj_lists
{7313, 5278}
building ratings_list
building other dicts


In [8]:
print(valid)
print(test)
print(train)

[[   383   4371      5      1]
 [  1022  56407      5      1]
 [   568  42538      5      1]
 ...
 [  7204 106306      3      1]
 [  4490  96000      2      0]
 [  1162  60977      3      0]]
[[ 3600 43485     5     1]
 [ 3130 87833     5     1]
 [  739 48162     5     1]
 ...
 [  511  2081     5     1]
 [ 1121 45177     3     0]
 [  698 46274     3     0]]
[[  7338 106685      5      1]
 [  4916   2742      4      1]
 [  6704   1965      5      1]
 ...
 [  3367  89336      4      1]
 [  4142  23291      5      1]
 [   548   2668      4      1]]


In [9]:
def build_item_adj_lists(history_v_lists):
    j = 0
    adj_lists = {}
    for key in history_v_lists.keys():
        adj_lists[key] = []
    for key in history_v_lists.keys():
        for key_temp in history_v_lists.keys():
            if key != key_temp and len(set(history_v_lists[key]) | set(history_v_lists[key_temp])) != 0:
                if len(set(history_v_lists[key]) & set(history_v_lists[key_temp])) / len(set(history_v_lists[key]) | set(history_v_lists[key_temp])) > 0.5:
                    adj_lists[key].append(key_temp)
                    j += 1
            if j > 300000:
                break
        if j > 300000:
            break            
            #print("添加item之间的连接")
    return adj_lists

print('validset')
validset = []
testset = []
for line in valid:
  user = line[0]
  item = line[1]
  rate = ratings_list[line[2]]
  label = line[3]
  if user in dic_user_o2i.keys():
    validset.append([dic_user_o2i[user], dic_item_o2i[item], rate, label])  
print('testset')
for line in test:
  user = line[0]
  item = line[1]
  rate = ratings_list[line[2]]
  label = line[3]
  if user in dic_user_o2i.keys():
    testset.append([dic_user_o2i[user], dic_item_o2i[item], rate, label]) 
print('trainingset')
for line in train:
  user = line[0]
  item = line[1]
  rate = ratings_list[line[2]]
  label = line[3]
  if user in dic_user_o2i.keys():
    history_u_lists[dic_user_o2i[user]].append(dic_item_o2i[item])
    history_ur_lists[dic_user_o2i[user]].append(rate)
    history_v_lists[dic_item_o2i[item]].append(dic_user_o2i[user])
    history_vr_lists[dic_item_o2i[item]].append(rate)
    data_total.append([dic_user_o2i[user], dic_item_o2i[item], rate, label])

print("line读取完成")
print(len(history_v_lists.keys()))
item_adj_lists = build_item_adj_lists(history_v_lists)
print("共享50%的user的item连接完成")

pickle_data = [history_u_lists, history_ur_lists, history_v_lists, history_vr_lists, data_total, validset, testset, social_adj_lists, item_adj_lists, ratings_list]
with open("data/ciao_rank1.pkl", 'wb') as fo:
    pickle.dump(pickle_data, fo)

validset
testset
trainingset
line读取完成
104975
共享50%的user的item连接完成
